In [1]:
%matplotlib widget
import argparse
import gc
import random
import torch
from PIL import Image
from torchvision.transforms import PILToTensor
from src.models.dift_sd import SDFeaturizer
from src.utils.visualization import Demo

c:\Users\Ludenbold\anaconda3\envs\PrakDiFT\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] Die angegebene Prozedur wurde nicht gefunden
  warn(f"Failed to load image Python extension: {e}")


In [2]:
torch.cuda.set_device(0)

In [3]:
dift = SDFeaturizer()

c:\Users\Ludenbold\anaconda3\envs\PrakDiFT\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
vae\diffusion_pytorch_model.safetensors not found
c:\Users\Ludenbold\anaconda3\envs\PrakDiFT\lib\site-packages\diffusers\models\modeling_utils.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to

In [4]:
# you can choose visualize cat or guitar
category = random.choice(['cat', 'guitar'])

print(f"let's visualize semantic correspondence on {category}")

let's visualize semantic correspondence on cat


In [5]:
if category == 'cat':
    filelist = ['./assets/cat.png', './assets/target_cat.png', './assets/target_cat.png']
elif category == 'guitar':
    filelist = ['./assets/guitar.png', './assets/target_guitar.png', './assets/target_guitar.png']

prompt = f'a photo of a {category}'

In [6]:
ft = []
imglist = []

# decrease these two if you don't have enough RAM or GPU memory
img_size = 768
ensemble_size = 8

for filename in filelist:
    img = Image.open(filename).convert('RGB')
    img = img.resize((img_size, img_size))
    imglist.append(img)
    img_tensor = (PILToTensor()(img) / 255.0 - 0.5) * 2
    ft.append(dift.forward(img_tensor,
                           prompt=prompt,
                           ensemble_size=ensemble_size))
ft = torch.cat(ft, dim=0)

gc.collect()
torch.cuda.empty_cache()

In [7]:
demo = Demo(imglist, ft, img_size)

In [8]:
'''
left is source image, right is target image.
you can click on the source image, and DIFT will find the corresponding
point on the right image, mark it with red point and also plot the per-pixel 
cosine distance as heatmap.
'''
demo.plot_img_pairs(fig_size=5)

: 